In this notebook I walk through the generation/inferece pipeline (spending most of the time on inference).

In [27]:
from abc import ABC, abstractmethod

import os
from tqdm import tqdm
import math 
import time

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

from scipy import stats

import torch
from torch.distributions import Beta
from torch.distributions.bernoulli import Bernoulli
from torch.nn.functional import log_softmax
from torch.optim import Adam, SGD
from torch.utils.data import DataLoader, TensorDataset

In [13]:
SEED = 1
np.random.seed(SEED)
torch.manual_seed(SEED)

### Generation:

Here I am using 2/10 generation.

In [14]:
### these params control the generation scheme
rho        = 0.8   # polarization
pop_size   = 50000 # num individuals
epsilon    = 0.05  # expected prop of speech consisting of neutral words
pi         = 0.5   # pi == 0.5 => beta mixture symmetrical (choose beta1 with prob pi = 0.5)
speech_len = 15    # words per speech

In [15]:
def generate(rho=rho, N=pop_size, epsilon=epsilon, pi=pi, speech_len=speech_len):
    """
    Uses 2/10 generation scheme to generate N samples.

    Returns:
        (X, y), (a, b, rho, epsilon, u)
        X.size() == [N, 3] is a vector of word counts
        y.size() == [N] is a vector of political parties
        u.shape  == (N,) is a vector of individual stances
        rho is true polarization
        epsilon is expected prop of neutral words
        a, b are true alpha/beta for beta mixture model
    """
    start = time.time()
    print(f'Beginning Data Generation...')
    print(f'=' * 20)
    
    ### get beta mixture model params 
    sigma = 0.175 * (rho ** 2) - 0.3625 * rho + 0.1875
    a     = rho * ((rho * (1 - rho)) / sigma - 1)
    b     = (1 - rho) * ((rho * (1 - rho)) / sigma - 1)

    print(f"True Alpha: {a}")
    print(f"True Beta: {b}")
    print(f"True Epsilon: {epsilon}\n")

    mean = a / (a + b)
    var  = a * b / ((a + b)**2 * (a + b + 1))

    if abs(mean - rho) > 10e-15:
        print(f'Mean: {mean}')
        print(f"Rho: {rho}")
        raise AssertionError(f"Mean of BMM params should be rho")

    if abs(var - sigma) > 10e-15:
        print(f'Var: {var}')
        print(f'Sigma: {sigma}')
        raise AssertionError(f"Var of BMM params should be sigma")

    ### u ~ pi Beta(a, b) + (1 - pi) Beta(b, a)
    weights = [pi, 1-pi]
    mixture_samples = np.random.choice([0, 1], size=N, p=weights)

    u = 2 * np.where(mixture_samples == 1, stats.beta.rvs(a, b, size=N), stats.beta.rvs(b, a, size=N)) - 1

    if u.shape != (N,):
        raise AssertionError(f"u.shape should be (N,)")

    print(f'mixture samples: {mixture_samples[:5]}')
    print(f'u samples: {u[:5]}\n')

    ### y = 1(u >= 0)
    y = (u >= 0).astype(int)
    print(f'y samples: {y[:5]}\n')

    ### phi is a prob matrix that is a function of u, epsilon
    phi = np.array([(1 - (u+1)/2) * (1 - epsilon), (u+1)/2 * (1 - epsilon), np.repeat(epsilon, pop_size)]).T
    print(f'phi samples:\n {phi[:5, :]}\n')

    if phi.shape != (N, 3):
        raise AssertionError(f'phi.shape should be (N, V) == (N, 3)')

    if abs(sum(phi[0]) - 1) > 10e-5:
        raise AssertionError(f'rows of phi should sum to 1')
    
    X = np.array([stats.multinomial.rvs(n=speech_len, p=phi[i, :]) for i in range(N)])
    print(f'X samples:\n {X[:5, :]}\n')
    
    if X.shape != (N, 3):
        raise AssertionError(f'X.shape should be (N, V) == (N, 3)')

    if X[:5].sum() != 15 * 5:
        raise AssertionError(f'rows of phi should sum to 1')

    X = torch.from_numpy(X).to(torch.float32)
    y = torch.from_numpy(y).to(torch.float32)

    known   = (X, y)
    unknown = (a, b, rho, epsilon, u)
    print('=' * 20)
    print(f'Generation Time: {round(time.time() - start, 3)} seconds for {N} samples.')
    return known, unknown

known, unknown = generate()
X, y = known
a, b, rho, epsilon, u = unknown

Beginning Data Generation...
True Alpha: 12.673684210526261
True Beta: 3.1684210526315644
True Epsilon: 0.05

mixture samples: [0 1 0 0 0]
u samples: [-0.71319006  0.57688516 -0.46523368 -0.74177243 -0.73809268]

y samples: [0 1 0 0 0]

phi samples:
 [[0.81376528 0.13623472 0.05      ]
 [0.20097955 0.74902045 0.05      ]
 [0.695986   0.254014   0.05      ]
 [0.8273419  0.1226581  0.05      ]
 [0.82559402 0.12440598 0.05      ]]

X samples:
 [[11  3  1]
 [ 2 12  1]
 [11  3  1]
 [14  0  1]
 [14  1  0]]

Generation Time: 1.479 seconds for 50000 samples.


### Inference

The goal of inference is to recover $u$ given a sample of $(x,y)$ pairs. We assume the following data generating process:
\begin{align*}
    u &\sim 2 \cdot \left(\pi\cdot\mathrm{Beta}\left(u;\alpha,\beta\right)+(1-\pi)\cdot\mathrm{Beta}\left(u;\beta,\alpha\right)\right)-1 \\
    y &= 1(u \ge \lambda) \\
    x &=\mathrm{Softmax}(Wu), W \in \R^V.
\end{align*}

For now, we assume $\lambda = 0, V = 3, \pi = 0.5$, so our parameters are $\theta = \{\alpha, \beta, W\}$. Given we assume this full form of $u$ parameterized by $\alpha, \beta$, recovering the distribution of $u$ is equivalent to recovering $\alpha, \beta$. This also gives us the following joint:

\begin{align*}

p(u,x^{(n)},y^{(n)};\theta) &= p(u)p(y^{(n)}|u)\prod_{s=1}^{S}p(x^{(n)}_s | u) \\
&= \left(\frac{1}{4}\mathrm{Beta}\left(\frac{u+1}{2};\alpha,\beta\right)+\frac{1}{4}\mathrm{Beta}\left(\frac{u+1}{2};\beta,\alpha\right)\right) \\&\cdot 1\left(1(u\ge0)=y^{(n)}\right)\frac{S!}{x^{(n)}_1!x^{(n)}_2!x^{(n)}_3!}\prod_{s=1}^{S}\mathrm{softmax}(Wu)_{x^{(n)}_s}

\end{align*}

The basic idea is to maximize the log marginal likelihood. Let $\theta = \{\alpha, \beta, W\}$. We'd like to find 
$$
\theta^* = \argmax_{\theta} \underbrace{\log p(x^{(1:N), y^{(1:N)}; \theta})}_{\mathcal{L}(\theta)}. 
$$


The goal is to recover alpha/beta (hence u given we are assuming u ~ 2[[1/2 Beta(a, b) + 1/2 Beta(b, a)] - 1) given (x,y) pairs.

To get MLE estimates for alpha, beta, W, we will take gradient descent steps on the log marginal likelihood. Its gradient is given by the following expression:
$$
\nabla_\theta \log p(x^{(1:N)},y^{(1:N)}) = \sum_{n=1}^{N} \mathbb{E}_{p(u|x^{(n)},y^{(n)};\theta)}[\nabla_\theta \log p(u,x^{(n)},y^{(n)};\theta)].
$$
There are a number of issues here, beginning with the fact that we cannot analytically recover the posterior due to the integral being intractable. However, since $u \in [-1,1]$, we can discretize the integral, which will be simpler than something like MCMC for sampling/using variational inference to approximate the posterior. To do this, let's write the gradient as follows:
$$
\nabla_\theta \log p(x^{(1:N)},y^{(1:N)}) = \sum_{n=1}^{N} \int_{u} \frac{p(x^{(n)}, y^{(n)}, u;\ \theta)}{p(x^{(n)},y^{(n)};\  \theta)}\nabla_\theta \log p(u,x^{(n)},y^{(n)};\theta) \mathrm{d}u
$$
To discretize, we simply need to assume that $u$ takes on $G$ linearlly spaced points in $[-1,1]$, rather than is continuous in $[-1,1]$, which gives us 
$$
\nabla_\theta \log p(x^{(1:N)},y^{(1:N)}) = \sum_{n=1}^{N} \sum_{u} \frac{p(x^{(n)}, y^{(n)}, u;\ \theta)}{p(x^{(n)},y^{(n)};\  \theta)}\nabla_\theta \log p(u,x^{(n)},y^{(n)};\theta) \mathrm{d}u
$$
It's convenient to move the gradient outside the sum, since then we can efficiently compute the double sum via matrices and then just sum the matrix/call .backward(). However, we cannot simply move the gradient outside unless we .detach() the "weight" (posterior probability).


In [22]:
grid_size  = 200 ### discretize integral into grid_size linearly spaced pts
batch_size = 100 ### num samples processed simultaneously
num_epochs = 200 ### number epochs
lr         = 0.1 ### learning rate

log_alpha = torch.normal(1.5, 1, size=(1,), requires_grad=True)
log_beta  = torch.normal(0, 1, size=(1,), requires_grad=True)
W         = torch.normal(3, 2, size=(3,), requires_grad=True)

print(f'Beginning Inference')
print(f'Initial Parameters:')
print(f'Alpha: {torch.exp(log_alpha).item()}')
print(f"Beta: {torch.exp(log_beta).item()}")
print(f"W: {W.data.tolist()}")

Beginning Inference
Initial Parameters:
Alpha: 0.9299120903015137
Beta: 0.8840445876121521
W: [10.173978805541992, -0.6625802516937256, 6.1974005699157715]


In [20]:
dataset    = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=batch_size)

X_batch, y_batch = list(dataloader)[0]

if list(X_batch.size()) != [batch_size, 3]:
    raise AssertionError(f"X_ex.size() should be [batch_size, V] == [batch_size, 3], got {X_batch.size()}")

if list(y_batch.size()) != [batch_size]:
    raise AssertionError(f"y_ex.size() should be [batch_size], got {y_batch.size()}")

In [23]:
def compute_nll(x_batch, y_batch, log_alpha, log_beta, W):
    """
    Parameters:
        (x_batch, y_batch) a batch of B (x,y) samples 
        (log_alpha, log_beta, W) trainable parameters

    Returns:
        A scaler quantity s.t. calling .backward() will compute
        grad_{theta} (L(theta))
    """
    raise NotImplementedError

def train(X, y, num_epochs=num_epochs, batch_size=batch_size, grid_size=grid_size, lr=lr):
    print(f'Beginning Training...')
    start = time.time()

    dataset    = TensorDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=batch_size)

    log_alpha = torch.normal(1.5, 1, size=(1,), requires_grad=True)
    log_beta  = torch.normal(0, 1, size=(1,), requires_grad=True)
    W         = torch.normal(3, 2, size=(3,), requires_grad=True)

    optimizer = SGD([log_alpha, log_beta, W], lr=lr)
    
    for epoch in tqdm(range(num_epochs)):
        print(f'Epoch {epoch+1} of {num_epochs}')
        ### only step after each epoch
        optimizer.zero_grad() 
        nll = 0
        for x_batch, y_batch in dataloader:
            nll += compute_nll(x_batch, y_batch, log_alpha, log_beta, W)
        
        nll = nll / len(dataloader.dataset)
        nll.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}, NLL: {round(nll.item(), 3)}, Alpha: {round(np.exp(log_alpha.item()),3)}, Beta: {round(np.exp(log_beta.item()),3)}, W: {W.tolist()}')
        print('=' * 20)

    final_alpha = np.exp(log_alpha.item())
    final_beta  = np.exp(log_beta.item())

    print(f"Training Took {round(time.time() - start, 2)} Seconds.")
    print(f"Trained Params: alpha: {final_alpha}, beta: {final_beta}, W: {W.tolist()}")

    return final_alpha, final_beta, W

Let's check how a few things work in torch.

In [32]:
"""
Checking if .log_prob() broadcasts across a tensor
"""
test = torch.tensor(
    [
    [1., 0., 1.],
    [1., 0., 1.]
    ]
)
test_dist = Bernoulli(torch.tensor([0.7]))
torch.exp(test_dist.log_prob(test))

tensor([[0.7000, 0.3000, 0.7000],
        [0.7000, 0.3000, 0.7000]])

In [49]:
"""
Given an NxM matrix and an Bx1 vector, create a tensor size
NxMxB which is the vector multiplied by each of the matrix values 
"""
test_u = torch.tensor(
    [
    [1., 2., 2.4],
    [1., 3., 1.1]
    ]
)
test_W = torch.tensor([1.,1.,2.])

test_res = torch.matmul(test_u.unsqueeze(2), test_W.unsqueeze(0).unsqueeze(0))

assert torch.all(test_res[0, 1] == torch.tensor([2., 2., 4.]))

In [70]:
test_res.size()

scaled_x_batch.unsqueeze(1) * log_softmax(softmax_input, dim=2)).sum(dim=2)

torch.Size([2, 3, 3])

In [77]:
test_res

tensor([[[1.0000, 1.0000, 2.0000],
         [2.0000, 2.0000, 4.0000],
         [2.4000, 2.4000, 4.8000]],

        [[1.0000, 1.0000, 2.0000],
         [3.0000, 3.0000, 6.0000],
         [1.1000, 1.1000, 2.2000]]])

In [79]:
ex = torch.tensor([[3., 7., 5.],
    [8., 3., 4.]])
print(log_softmax(test_res, dim=2))
ex.unsqueeze(1) * log_softmax(test_res, dim=2)

tensor([[[-1.5514, -1.5514, -0.5514],
         [-2.2395, -2.2395, -0.2395],
         [-2.5667, -2.5667, -0.1667]],

        [[-1.5514, -1.5514, -0.5514],
         [-3.0949, -3.0949, -0.0949],
         [-1.6103, -1.6103, -0.5103]]])


tensor([[[ -4.6543, -10.8601,  -2.7572],
         [ -6.7186, -15.6768,  -1.1977],
         [ -7.7002, -17.9671,  -0.8337]],

        [[-12.4116,  -4.6543,  -2.2058],
         [-24.7594,  -9.2848,  -0.3797],
         [-12.8822,  -4.8308,  -2.0411]]])

In [55]:
"""
Numerically stable factorial
Uses gamma function
"""
factorial = lambda x : torch.exp(torch.lgamma(x+1))

assert factorial(torch.tensor(1.)) == torch.tensor(1.)
assert factorial(torch.tensor(2.)) == torch.tensor(2.)
assert factorial(torch.tensor(3.)) == torch.tensor(6.)
print(f"2.9999! = {factorial(torch.tensor(2.9999))}")

2.9999! = 5.999247074127197


In [64]:
test_x_batch

tensor([[11.,  3.,  1.],
        [ 2., 12.,  1.],
        [11.,  3.,  1.],
        [14.,  0.,  1.],
        [14.,  1.,  0.]])

In [66]:
test_x_batch = X_batch[:5]
print(test_x_batch)

x_factorial = factorial(test_x_batch)
print(torch.log(x_factorial))
print(torch.log(x_factorial.sum(dim=1, keepdim=True)))
"""
x_batch * factorial(x_batch) 
    x_factorial = factorial(x_batch)

    sum_x_factorial = x_factorial.sum(dim=1, keepdim=True)

    scaling_factor = x_factorial / sum_x_factorial

    scaled_x_batch = x_batch * scaling_factor
"""

tensor([[11.,  3.,  1.],
        [ 2., 12.,  1.],
        [11.,  3.,  1.],
        [14.,  0.,  1.],
        [14.,  1.,  0.]])
tensor([[17.5023,  1.7918,  0.0000],
        [ 0.6931, 19.9872,  0.0000],
        [17.5023,  1.7918,  0.0000],
        [25.1912,  0.0000,  0.0000],
        [25.1912,  0.0000,  0.0000]])
tensor([[17.5023],
        [19.9872],
        [17.5023],
        [25.1912],
        [25.1912]])


'\nx_batch * factorial(x_batch) \n    x_factorial = factorial(x_batch)\n\n    sum_x_factorial = x_factorial.sum(dim=1, keepdim=True)\n\n    scaling_factor = x_factorial / sum_x_factorial\n\n    scaled_x_batch = x_batch * scaling_factor\n'

In [90]:
jamie = torch.tensor([
    [1.1, 2., 3.4],
    [4.2, 5.1, 6.2]
])
jamie - torch.log(factorial(X_batch[:2])).sum(dim=1, keepdim=True)

tensor([[-18.1941, -17.2941, -15.8941],
        [-16.4804, -15.5804, -14.4804]])

In [95]:
def compute_log_joint(u_mat, x_batch, y_batch, log_alpha, log_beta, W, constants=True):
    """
    Parameters:
        x_batch (size [batch_size, 3])
        y_batch (size [batch_size])
        u_mat   (size [batch_size, N])
        (log_alpha, log_beta, W) trainable params 
        constants (bool) true iff include constants
    
    constants are necessary for computing joint probabilities exactly but not for gradient expressions
    
    Returns:
        torch.tensor (size [batch_size, N]), where the (i,j)th element 
        is the log joint probability (log p(u, x, y; theta)). Here u is the (i,j)th
        element of u_mat, x is the ith row of x_batch, y is the ith element of y_batch

    """
    ### log prior: log p(u) = log(1/4 Beta((u+1)/2; a, b) + 1/4 Beta((u+1)/2; b, a))
    alpha = torch.exp(log_alpha)
    beta  = torch.exp(log_beta)
        
    beta_dist_ab = Beta(alpha, beta)
    beta_dist_ba = Beta(beta, alpha)

    log_beta_prob_ab = beta_dist_ab.log_prob((u_mat + 1) / 2)
    log_beta_prob_ba = beta_dist_ba.log_prob((u_mat + 1) / 2)

    prior = torch.log(torch.exp(log_beta_prob_ab) + torch.exp(log_beta_prob_ba)) + torch.log(torch.tensor(0.25))

    assert prior.size() == u_mat.size()
  
    ### log likelihood: log p(x, y | u) = dot(x, log Softmax(Wu)) (assuming y is compatible)
    W_expanded = W.unsqueeze(0).unsqueeze(0)
    assert list(W_expanded.size()) == [1,1,3]
    u_expanded = u_mat.unsqueeze(2)

    softmax_input = torch.matmul(u_expanded, W_expanded)  ### size BxNx3
    likelihood = (x_batch.unsqueeze(1) * log_softmax(softmax_input, dim=2)).sum(dim=2) ### size BxN
    
    if constants:
        likelihood += torch.log(factorial(torch.tensor(speech_len)))  ### log(S!)
        likelihood -= torch.log(factorial(x_batch)).sum(dim=1, keepdim=True) # - log(x_1!) -log(x_2!) - log(x_3!)

    # x_factorial = factorial(x_batch)

    # sum_x_factorial = x_factorial.sum(dim=1, keepdim=True)

    # scaling_factor = x_factorial / sum_x_factorial

    # scaled_x_batch = x_batch * scaling_factor

    # x_batch_log_prob = (scaled_x_batch.unsqueeze(1) * log_softmax(softmax_input, dim=2)).sum(dim=2)
   
    #assert x_batch_log_prob.size() == beta_log_prob.size()
    return prior + likelihood


def compute_nll(x_batch, y_batch, log_alpha, log_beta, W):
    """
    Parameters:
        x_batch (size [batch_size, 3])
        y_batch (size [batch_size])
    
    Returns:
        torch.tensor (size [batch_size, approx_size]), where the ith row are 
        samples from the approximate posterior p(u | x^(i), y^(i); theta).
        Here, x^(i), y^(i) are the ith row of x_batch/y_batch. 
    """
   # Initialize u_mat based on conditions
    u_mat = torch.empty(batch_size, grid_size, device=x_batch.device)
    u_mat[y_batch == 1] = torch.linspace(1/(grid_size+1), 1-1/(grid_size+1), grid_size).repeat((y_batch == 1).sum(), 1)
    u_mat[y_batch == 0] = torch.linspace(-1+1/(grid_size+1), -1/(grid_size+1), grid_size).repeat((y_batch == 0).sum(), 1)

    assert list(u_mat.size()) == [x_batch.size()[0], grid_size]

    # Compute log_joint with u_mat, x_batch, and y_batch
    log_joint = compute_log_joint(u_mat, x_batch, y_batch, log_alpha, log_beta, W)

    # Compute the posterior probabilities but detach to prevent gradients through this path
    max_log_prob = torch.max(log_joint, dim=1, keepdim=True)[0]
    joint_probs = torch.exp(log_joint - max_log_prob)
    posterior = joint_probs / joint_probs.sum(dim=1, keepdim=True)  # Do not detach yet

    # Compute the weighted negative log likelihood (NLL)
    # Detach posterior here to use it as a constant weight for log_joint contributions
    weighted_log_joint = posterior.detach() * log_joint
    nll = -weighted_log_joint.sum()

    return nll

In [96]:
train(X, y)

Beginning Training...


  0%|          | 0/200 [00:00<?, ?it/s]

Epoch 1 of 200


  0%|          | 1/200 [00:07<25:21,  7.64s/it]

Epoch 1, NLL: 21.233, Alpha: 8.47, Beta: 0.222, W: [4.713240146636963, 3.1470117568969727, 1.3748860359191895]
Epoch 2 of 200


...

Epoch 128, NLL: 5.605, Alpha: 2.215, Beta: 0.831, W: [1.1986161470413208, 4.9617414474487305, 3.074781656265259]
Epoch 129 of 200


 64%|██████▍   | 129/200 [11:55<06:08,  5.20s/it]

Epoch 129, NLL: 5.605, Alpha: 2.217, Beta: 0.832, W: [1.1975399255752563, 4.962820053100586, 3.0747792720794678]
Epoch 130 of 200


 65%|██████▌   | 130/200 [12:04<07:31,  6.44s/it]

Epoch 130, NLL: 5.605, Alpha: 2.22, Beta: 0.834, W: [1.1964768171310425, 4.963885307312012, 3.0747768878936768]
Epoch 131 of 200
